We will use the Minimum Daily Temperatures dataset as an example. This dataset
describes the minimum daily temperatures over 10 years (1981-1990) in the city Melbourne,
Australia.

# Date Time Features

# Lag Features

# Rolling Window Statistics

# Expanding Window Statistics